In [1]:
from collections import defaultdict
from copy import deepcopy
import numpy as np
import pandas as pd
import cv2
import os
import os.path as osp
import torch
from pathlib import Path

from PIL import Image
from tqdm import tqdm

from depth_anything_v2.dpt import DepthAnythingV2
from ioutils import load_pickle

%load_ext autoreload
%autoreload 2


xFormers not available
xFormers not available


## Guide
- Install Depth-Anything accordingly to [the instructions](https://huggingface.co/depth-anything/Depth-Anything-V2-Large).

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Large
model = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
model.load_state_dict(torch.load('checkpoints/depth_anything_v2_vitl.pth', map_location='cpu'))

# Base
# model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
# model.load_state_dict(torch.load('checkpoints/depth_anything_v2_vitb.pth', map_location='cpu'))

# Small
# model = DepthAnythingV2(encoder='vits', features=64, out_channels=[48, 96, 192, 384])
# model.load_state_dict(torch.load('checkpoints/depth_anything_v2_vits.pth', map_location='cpu'))

model.eval().to(device)

# raw_img = cv2.imread('/home/marek/Work/COOOL_benchmark/coool/experiment/assets/examples/demo01.jpg')
# raw_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)
# raw_img = torch.from_numpy(raw_img).unsqueeze(0).to(device)
# depth = model.infer_image(raw_img) # HxW raw depth map
# Image.fromarray(depth).show()

/tmp/ipykernel_90289/2270525560.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('checkpoints/depth_anything_v2_vitl.pth', map_location='

DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-23): 24 x NestedTensorBlock(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (drop): Dropout(p=0.0, inplace=Fal

In [3]:
annotations_path = "../data/annotations_public.pkl"
annotations = load_pickle(annotations_path)
video_root = "/home/marek/datasets/coool-benchmark"


In [4]:

video_names = sorted(list(annotations.keys()))
for video_name in tqdm(video_names, total=len(video_names)):

    video_path = osp.join(video_root, f"{video_name}.mp4")

    video_name = Path(video_path).stem
    video_data = []
    video_stream = cv2.VideoCapture(video_path)
    fps = video_stream.get(cv2.CAP_PROP_FPS)
    if not video_stream.isOpened():
        raise ValueError(f"Video {video_name} could not be opened.")

    frame = 0
    while video_stream.isOpened():
        ret, frame_image = video_stream.read()
        if not ret:
            break
        if frame % fps == 0:
            depth = model.infer_image(frame_image)
            # depth_normalized = depth / depth.max() * 255
            # depth_captures[video_name][frame] = depth_normalized.astype(np.uint8)
            img_path = osp.join("../results/depth_captures", f"{video_name}_{frame}.jpeg")
            cv2.imwrite(img_path, depth)

        frame += 1
    video_stream.release()


100%|██████████| 200/200 [06:17<00:00,  1.89s/it]
